In [1]:
%cd ~/src/second-best-bench/
from src.plots.common import *
import numpy as np

/home/yimingz3/src/second-best-bench


In [2]:
def compute_summary(df):
    
    summary = df.groupby(['model_family', 'model_alias']).agg(
        mean_distinct=('distinct', 'mean'),
        mean_utility=('utility', 'mean')
    ).reset_index()
    
    summary['distinct'] = summary['mean_distinct'] + 1
    
    return summary[['model_family', 'model_alias', 'distinct', 'mean_utility']]

def model_scores_at_patience(patience):
    score_dfs = []
    for subset in ["wildchat", "curated"]:
        for model_family, model_path, model_alias in models:
            df = pd.read_json(f"eval/{subset}/{model_path}/scores.jsonl", lines=True)
            df = df.drop(columns="utility")
            df["subset"] = subset
            if patience == 0.0:
                df["utility"] = df["generation_scores"].map(lambda l: l[0])    
            else:
                df["utility"] = df["generation_scores"].map(lambda l: np.average(l, weights=patience ** np.arange(len(l))))
            df["model_family"] = model_family
            df["model_alias"] = model_alias
            score_dfs.append(df)

    model_scores = pd.concat(score_dfs)
    model_scores["distinct"] = model_scores["partition_scores"].map(len)
    model_scores["patience"] = patience
    return model_scores

In [3]:
all_model_scores = pd.concat([model_scores_at_patience(p) for p in np.linspace(0.0, 1.0, 11)])

In [4]:
def compute_summary(df):
    
    summary = df.groupby(['model_family', 'model_alias', 'patience']).agg(
        mean_distinct=('distinct', 'mean'),
        mean_utility=('utility', 'mean')
    ).reset_index()
    
    summary['distinct'] = summary['mean_distinct'] + 1
    
    return summary[['model_family', 'model_alias', 'distinct', 'mean_utility', 'patience']]

eval_data = compute_summary(all_model_scores)
eval_data = eval_data.rename(columns={'mean_utility': 'utility'})


In [5]:
eval_data[eval_data["model_family"].isin(["OpenAI", "Anthropic"])]

,model_family,model_alias,distinct,utility,patience
0,Anthropic,Claude 3 Opus,3.038182,8.287273,0.0
1,Anthropic,Claude 3 Opus,3.038182,7.616591,0.1
2,Anthropic,Claude 3 Opus,3.038182,6.935912,0.2
3,Anthropic,Claude 3 Opus,3.038182,6.242868,0.3
4,Anthropic,Claude 3 Opus,3.038182,5.534868,0.4
5,Anthropic,Claude 3 Opus,3.038182,4.810700,0.5
6,Anthropic,Claude 3 Opus,3.038182,4.075431,0.6
7,Anthropic,Claude 3 Opus,3.038182,3.348409,0.7
8,Anthropic,Claude 3 Opus,3.038182,2.667487,0.8
9,Anthropic,Claude 3 Opus,3.038182,2.078907,0.9


In [6]:
llama_patience_chart = alt.Chart(eval_data[eval_data["model_family"].isin(["Llama"])]).mark_line(point=True).encode(
    x=alt.X('patience:Q', title='Patience'),
    y=alt.Y('mean(utility):Q', title='Utility'),
    color=alt.Color('model_alias:O', title="Model", sort=model_order)
).properties(
    width=200,
    height=200
)

llama_patience_chart.save("plots/llama-patience.json")
llama_patience_chart

alt.Chart(...)

In [8]:
gemma_patience_chart = alt.Chart(eval_data[eval_data["model_family"].isin(["Gemma"])]).mark_line(point=True).encode(
    x=alt.X('patience:Q', title='Patience'),
    y=alt.Y('mean(utility):Q', title='Utility'),
    color=alt.Color('model_alias:O', title="Model", sort=model_order)
).properties(
    width=200,
    height=200
)

gemma_patience_chart.save("plots/gemma-patience.json")
gemma_patience_chart

alt.Chart(...)